Minimal character-level Vanilla RNN model. 

Originally written by Andrej Karpathy (@karpathy)
See: https://gist.github.com/karpathy/d4dee566867f8291f086

Some details were tweaked by me to fit into Python3 and I add some comments to make it easy to understand. Also predict() was implemented.

In [7]:
import numpy as np

In [19]:
# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.'% (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1557 characters, 58 unique.


In [20]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In the code, each variable represents...

- h : the output at each time step
- y : final scores 

and we feed input as a one-hot vector 'cause Wxh is of shape (..., vocab_size)

In [21]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        # dy: dL/df 
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        # dWhy: dL/dWhy_ij = dL/df_i * hs[t]_j
        dWhy += np.dot(dy, hs[t].T)
        # dby: dL/df * 1
        dby += dy
        # dh: dL/dhs[t]_i = (\sum_{j=0}^{j<vocab_size} Why_ji * dy_j) + (gradient flow from the next layer)
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        # dhraw := dL/dh'
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        # dbh: dL/dh' * 1
        dbh += dhraw
        # dWxh, dWhh: similar to dWhy
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        # dhnext: dL/dh[t-1] whose calculation is similar to dh
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

# This function is called when you make a prediction.

def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model made of softmax values
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

# Training

In [28]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

iteration = 30000

for i in range(iteration):
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    # if n % 100 == 0:
    #     sample_ix = sample(hprev, inputs[0], 200)
    #     txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    #     print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 200 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                  [dWxh, dWhh, dWhy, dbh, dby], 
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter          

iter 0, loss: 101.410282
iter 200, loss: 98.115715
iter 400, loss: 90.028837
iter 600, loss: 81.686995
iter 800, loss: 73.898432
iter 1000, loss: 66.715745
iter 1200, loss: 60.215532
iter 1400, loss: 54.405039
iter 1600, loss: 49.246946
iter 1800, loss: 44.736955
iter 2000, loss: 40.778748
iter 2200, loss: 37.319262
iter 2400, loss: 34.175291
iter 2600, loss: 31.408708
iter 2800, loss: 28.934205
iter 3000, loss: 26.729003
iter 3200, loss: 24.750248
iter 3400, loss: 23.238156
iter 3600, loss: 21.763793
iter 3800, loss: 20.395916
iter 4000, loss: 19.164656
iter 4200, loss: 18.015122
iter 4400, loss: 16.990547
iter 4600, loss: 16.113048
iter 4800, loss: 15.331188
iter 5000, loss: 14.532514
iter 5200, loss: 13.717501
iter 5400, loss: 13.074833
iter 5600, loss: 12.682093
iter 5800, loss: 12.075024
iter 6000, loss: 11.510866
iter 6200, loss: 10.962611
iter 6400, loss: 10.612205
iter 6600, loss: 10.243169
iter 6800, loss: 9.838026
iter 7000, loss: 9.446223
iter 7200, loss: 9.019860
iter 7400,

# Prediction

Citation from the original author's blog:
        http://karpathy.github.io/2015/05/21/rnn-effectiveness/


At test time, we feed a character into the RNN and 
get a distribution over what characters are likely to come next. 
We sample from this distribution, and feed it right back in to get the next letter.
Repeat this process and you are sampling text! 

In [29]:
def predict(char, length=2000):
    idx = char_to_ix[char]
    hprev = np.zeros((hidden_size,1))
    sample_ix = sample(hprev, inputs[0], length)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    return txt

def output_to_file(txt):
    f = open('output.txt', 'w')
    f.write(txt)

prediction = predict('#')
output_to_file(prediction)